In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

In [144]:
textosRDD = sc.parallelize([(1,'fernet pelota casa futbol perro hamburguesa tren alcohol fernet tomate frutilla'), 
                            (2,'casa arbol cancha cerveza river boca playa colegio cigarrillo'), 
                            (3,'colegio alcohol mate europa viajar fiesta boliche fernet tren auto avion comida'), 
                            (4,'palicula cine futbol tomate lechuga colegio cerveza futbol rugby tren auto cigarrillo'), 
                            (5,'river river boca gato fiesta cerveza mate boliche avion comida auto')])

usuariosRDD = sc.parallelize([(1,'fernet',10), (1,'pelota',10), (1,'fiesta',6), (2,'river',10), (2,'cerveza',8), 
                             (1,'boca',-2), (2,'boca',-10), (1,'cigarrillo',-5), (2,'cigarrillo',2), 
                             (1,'futbol',4), (2,'futbol',8), (1,'mate',-7), (2,'mate',5), (1,'auto',9), 
                             (2,'auto',2), (1,'colegio',-6), (2,'colegio',-1), (1,'gato',5), (2,'perro',8)])

In [145]:
textosRDD = textosRDD.map(lambda x: (x[0],x[1].split())) #[(docIDs,['palabra1',...,'palabraN'])]
def f(x): return x
textosRDD = textosRDD.flatMapValues(f) #me une la clave (docID) con cada una de las palabras de su lista (valores)
textosRDD = textosRDD.map(lambda x: (x[1],x[0]))
textosRDD.collect()

[('fernet', 1),
 ('pelota', 1),
 ('casa', 1),
 ('futbol', 1),
 ('perro', 1),
 ('hamburguesa', 1),
 ('tren', 1),
 ('alcohol', 1),
 ('fernet', 1),
 ('tomate', 1),
 ('frutilla', 1),
 ('casa', 2),
 ('arbol', 2),
 ('cancha', 2),
 ('cerveza', 2),
 ('river', 2),
 ('boca', 2),
 ('playa', 2),
 ('colegio', 2),
 ('cigarrillo', 2),
 ('colegio', 3),
 ('alcohol', 3),
 ('mate', 3),
 ('europa', 3),
 ('viajar', 3),
 ('fiesta', 3),
 ('boliche', 3),
 ('fernet', 3),
 ('tren', 3),
 ('auto', 3),
 ('avion', 3),
 ('comida', 3),
 ('palicula', 4),
 ('cine', 4),
 ('futbol', 4),
 ('tomate', 4),
 ('lechuga', 4),
 ('colegio', 4),
 ('cerveza', 4),
 ('futbol', 4),
 ('rugby', 4),
 ('tren', 4),
 ('auto', 4),
 ('cigarrillo', 4),
 ('river', 5),
 ('river', 5),
 ('boca', 5),
 ('gato', 5),
 ('fiesta', 5),
 ('cerveza', 5),
 ('mate', 5),
 ('boliche', 5),
 ('avion', 5),
 ('comida', 5),
 ('auto', 5)]

In [146]:
usuariosRDD = usuariosRDD.map(lambda x: (x[1],(x[0],x[2])))
usuariosRDD.collect()

[('fernet', (1, 10)),
 ('pelota', (1, 10)),
 ('fiesta', (1, 6)),
 ('river', (2, 10)),
 ('cerveza', (2, 8)),
 ('boca', (1, -2)),
 ('boca', (2, -10)),
 ('cigarrillo', (1, -5)),
 ('cigarrillo', (2, 2)),
 ('futbol', (1, 4)),
 ('futbol', (2, 8)),
 ('mate', (1, -7)),
 ('mate', (2, 5)),
 ('auto', (1, 9)),
 ('auto', (2, 2)),
 ('colegio', (1, -6)),
 ('colegio', (2, -1)),
 ('gato', (1, 5)),
 ('perro', (2, 8))]

In [147]:
docFinalRDD = usuariosRDD.join(textosRDD) # Une las claves que son iguales formando una tupla de sus valores.
# Por lo tanto nos quedaran solamente las palabras para las cuales los usuarios tienen score.
docFinalRDD.collect()

[('auto', ((1, 9), 3)),
 ('auto', ((1, 9), 4)),
 ('auto', ((1, 9), 5)),
 ('auto', ((2, 2), 3)),
 ('auto', ((2, 2), 4)),
 ('auto', ((2, 2), 5)),
 ('gato', ((1, 5), 5)),
 ('river', ((2, 10), 2)),
 ('river', ((2, 10), 5)),
 ('river', ((2, 10), 5)),
 ('mate', ((1, -7), 3)),
 ('mate', ((1, -7), 5)),
 ('mate', ((2, 5), 3)),
 ('mate', ((2, 5), 5)),
 ('fiesta', ((1, 6), 3)),
 ('fiesta', ((1, 6), 5)),
 ('pelota', ((1, 10), 1)),
 ('boca', ((1, -2), 2)),
 ('boca', ((1, -2), 5)),
 ('boca', ((2, -10), 2)),
 ('boca', ((2, -10), 5)),
 ('perro', ((2, 8), 1)),
 ('colegio', ((1, -6), 2)),
 ('colegio', ((1, -6), 3)),
 ('colegio', ((1, -6), 4)),
 ('colegio', ((2, -1), 2)),
 ('colegio', ((2, -1), 3)),
 ('colegio', ((2, -1), 4)),
 ('cerveza', ((2, 8), 2)),
 ('cerveza', ((2, 8), 4)),
 ('cerveza', ((2, 8), 5)),
 ('cigarrillo', ((1, -5), 2)),
 ('cigarrillo', ((1, -5), 4)),
 ('cigarrillo', ((2, 2), 2)),
 ('cigarrillo', ((2, 2), 4)),
 ('futbol', ((1, 4), 1)),
 ('futbol', ((1, 4), 4)),
 ('futbol', ((1, 4), 4)),
 

In [148]:
docFinalRDD = docFinalRDD.values()
docFinalRDD = docFinalRDD.map(lambda x: (x[0][0],x[1],x[0][1])) # queda (userid,docid,score)
docFinalRDD.collect()

[(1, 3, 9),
 (1, 4, 9),
 (1, 5, 9),
 (2, 3, 2),
 (2, 4, 2),
 (2, 5, 2),
 (1, 5, 5),
 (2, 2, 10),
 (2, 5, 10),
 (2, 5, 10),
 (1, 3, -7),
 (1, 5, -7),
 (2, 3, 5),
 (2, 5, 5),
 (1, 3, 6),
 (1, 5, 6),
 (1, 1, 10),
 (1, 2, -2),
 (1, 5, -2),
 (2, 2, -10),
 (2, 5, -10),
 (2, 1, 8),
 (1, 2, -6),
 (1, 3, -6),
 (1, 4, -6),
 (2, 2, -1),
 (2, 3, -1),
 (2, 4, -1),
 (2, 2, 8),
 (2, 4, 8),
 (2, 5, 8),
 (1, 2, -5),
 (1, 4, -5),
 (2, 2, 2),
 (2, 4, 2),
 (1, 1, 4),
 (1, 4, 4),
 (1, 4, 4),
 (2, 1, 8),
 (2, 4, 8),
 (2, 4, 8),
 (1, 1, 10),
 (1, 1, 10),
 (1, 3, 10)]

In [149]:
docFinalRDD = docFinalRDD.map(lambda x: ((x[0],x[1]),x[2]))
docFinalRDD = docFinalRDD.reduceByKey(lambda x,y: x+y)
docFinalRDD = docFinalRDD.map(lambda x: (x[0][0], x[0][1], x[1])) #(userid,docid,scoreTotal)
docFinalRDD.collect()

[(1, 5, 11),
 (1, 2, -13),
 (1, 3, 12),
 (2, 5, 25),
 (2, 4, 27),
 (1, 1, 34),
 (2, 3, 6),
 (2, 2, 9),
 (2, 1, 16),
 (1, 4, 6)]